# Simple diet optimization example

#### Objective

Our objective is to build a meal that has the following characteristics:
-   Contains 25g of proteins
- Contains 10g of lipids
- Contains 80g of glucids

for a minimal cost

#### Food at disposition

In order to build that, meal we have (with all the macro nutriments & costs expressed for 100g of the element)
- Meat: 25g of proteins, 0g of lipids, and 0g of glucids, cost 5€
- Pasta: 0g of proteins, 0g of lipids, and 23.4g of glucids, cost 2€
- Tomato Sauce: 0g of proteins, 0g of lipids, and 7.5g of glucids, cost 3€
- Olive Oil: 0g of proteins, 99.9g of lipids, and 0g of glucids, cost 8€

##### Mathematical writing of the objective function
The Meal features ($M$) is a function of the decision variables ($w_{x_i}$) which are the quantity of each element (${x_i}$) that we put in the meal, and ${x_{i_P}}$, ${x_{i_L}}$, ${x_{i_G}}$, ${x_{i_C}}$ the respective macro nutriment composition in proteins, lipids, glucids, and cost for each element - hence we can write :
$$

M = 

\left[\begin{array}{cc} 
w_{x_1}\\
w_{x_2}\\
w_{x_3}\\
w_{x_4}\\
\end{array}\right]^T
\left[\begin{array}{cc} 
x_{1_P} & x_{1_L} & x_{1_G} & x_{1_C}\\ 
x_{2_P} & x_{2_L} & x_{2_G} & x_{2_C}\\
x_{3_P} & x_{3_L} & x_{3_G} & x_{3_C}\\
x_{4_P} & x_{4_L} & x_{4_G} & x_{4_C}\\
\end{array}\right]
$$

Our objective function & constraints are then, with $t_P$, $t_L$, $t_G$, our target nutriments:
$$ 
\text{minimize }   w_{x_1}x_{1_C} + w_{x_2}x_{2_C} + w_{x_3}x_{3_C} + w_{4_1}x_{4_C} \\
\textrm{subject to :} \\
w_{x_1}, w_{x_2}, w_{x_3}, w_{x_4} > 0 \\
{w_{x_1}x_{1_P} + w_{x_2}x_{2_P} + w_{x_3}x_{3_P} + w_{4_1}x_{4_P}} > t_P \\
{w_{x_1}x_{1_L} + w_{x_2}x_{2_L} + w_{x_3}x_{3_L} + w_{4_1}x_{4_L}} > t_L \\
{w_{x_1}x_{1_G} + w_{x_2}x_{2_G} + w_{x_3}x_{3_G} + w_{4_1}x_{4_G}} > t_G \\
$$

So if we write with the actual constants it's expresssed as:
$$
\text{minimize }   5{w_{x_1} + 2w_{x_2} + 3w_{x_3} + 8w_{x_4}}\\
\textrm{subject to :} \\
w_{x_1}, w_{x_2}, w_{x_3}, w_{x_4} > 0 \\
{25w_{x_1}} > 25 \\
{99.9w_{x_4}} > 10 \\
{23.4w_{x_2} + 7.5w_{x_3}} > 80 \\
$$

This is a classic linear optimization problem.

Let's solve it with CVXPY

##### Imports

In [1]:
from cvxopt import matrix, solvers
from IPython.display import display
import numpy as np
import pandas as pd


In [2]:
pd.options.display.float_format = '{:,.2f}'.format

##### Problem data definition

In [3]:
minimum_nutriments_df = pd.DataFrame({
    "Protein": {"Target": 25.},
    "Lipid": {"Target": 10.},
    "Glucid": {"Target": 80.} 
})
macro_nutriments_costs_df = pd.DataFrame({
    "Protein": {"Meat": 25., "Pasta": 0., "TomatoSauce": 0., "OliveOil": 0.},
    "Lipid": {"Meat": 0., "Pasta": 0., "TomatoSauce": 0., "OliveOil": 99.9},
    "Glucid": {"Meat": 0., "Pasta": 23.4, "TomatoSauce": 7.5, "OliveOil": 0.},
    "Cost": {"Meat": 5., "Pasta": 2., "TomatoSauce": 3., "OliveOil": 8.}
})

In [4]:
# Define A Matrix
macro_nutriments_df = macro_nutriments_costs_df.drop(columns="Cost")
positivity_constraint = pd.DataFrame(-np.identity(4))
positivity_constraint.columns = macro_nutriments_df.index
positivity_constraint = positivity_constraint.add_prefix("PosConstraint_")
positivity_constraint.index = macro_nutriments_df.index
A_df = pd.concat([-macro_nutriments_df, positivity_constraint], axis=1).transpose()

In [5]:
# Define b Matrix
b_df = minimum_nutriments_df
pos_constraint_row = pd.DataFrame({column: {"Target":0} for column in positivity_constraint.columns})
b_df = -pd.concat([b_df, pos_constraint_row], axis=1).transpose()

In [6]:
# Define c Matrix (our cost function)
c_df = macro_nutriments_costs_df[["Cost"]]

In [7]:
# Solve the optimization problem
A = matrix(np.array(A_df))
b = matrix(np.array(b_df))
c = matrix(np.array(c_df))
sol=solvers.lp(c,A,b)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.4959e+01  2.0830e+02  2e+01  4e-02  2e+01  1e+00
 1:  1.2668e+01  2.5752e+01  8e-01  2e-03  1e+00  2e-01
 2:  1.2652e+01  1.3308e+01  4e-02  1e-04  6e-02  6e-03
 3:  1.2639e+01  1.2645e+01  3e-04  1e-06  6e-04  6e-05
 4:  1.2638e+01  1.2638e+01  3e-06  1e-08  6e-06  6e-07
 5:  1.2638e+01  1.2638e+01  3e-08  1e-10  6e-08  6e-09
Optimal solution found.


In [8]:
# Display solution
sol_df = pd.DataFrame(sol["x"]) * 100
sol_df.index = macro_nutriments_costs_df.index
sol_df.columns = ["Quantity (g)"]
obj_df = pd.DataFrame({"Cost (€)": {"Cost (€)": sol["primal objective"]}})
display(sol_df)
display(obj_df)

,Quantity (g)
Meat,100.00
Pasta,341.88
TomatoSauce,-0.00
OliveOil,10.01


,Cost (€)
Cost (€),12.64


#### First Solution

We see that, although we have built a meal that has the minimum cost in line with our constraints in terms of nutriments, our meal contains no sauce.
This makes sense, given that 1g of pasta gives more glucids than 1g of sauce, for an inferior cost.
However, in terms of taste, this isn't optimal, we would ideally like to have some sauce in our meal.
Actually we would want the same quantity of pasta of the same quantity of sauce.
We can write the following constraint like this:
$$w_{x_2}  = w_{x_3}$$
or, in constraint optimization terms:
$$w_{x_2} - w_{x_3} = 0$$

In [9]:
# Add one more constraint in our A matrix
pasta_sauce_constraint = pd.DataFrame({"Meat": {"EqConstraint_Meat_Pasta": 0}, "Pasta":{"EqConstraint_Meat_Pasta": 1}, "TomatoSauce": {"EqConstraint_Meat_Pasta": -1}, "OliveOil": {"EqConstraint_Meat_Pasta": 0}})
A_df_new_constraint = pd.concat([A_df, pasta_sauce_constraint], axis=0)

In [10]:
# Reflect that in our b Matrix
eq_constraint_row = pd.DataFrame({"Target": {"EqConstraint_Meat_Pasta": 0}})
b_df_new_constraint = pd.concat([b_df, eq_constraint_row])

In [11]:
# Solve the optimization problem
A = matrix(np.array(A_df_new_constraint))
b = matrix(np.array(b_df_new_constraint))
c = matrix(np.array(c_df))
sol=solvers.lp(c,A,b)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.7258e+01  1.6223e+02  3e+01  6e-02  1e+01  1e+00
 1:  1.8836e+01  3.2338e+01  1e+00  6e-03  1e+00  3e-01
 2:  1.8757e+01  1.9470e+01  5e-02  3e-04  7e-02  1e-02
 3:  1.8746e+01  1.8754e+01  6e-04  3e-06  7e-04  1e-04
 4:  1.8746e+01  1.8746e+01  6e-06  3e-08  7e-06  1e-06
 5:  1.8746e+01  1.8746e+01  6e-08  3e-10  7e-08  1e-08
Optimal solution found.


In [12]:
# Display solution
sol_df = pd.DataFrame(sol["x"]) * 100
sol_df.index = macro_nutriments_costs_df.index
sol_df.columns = ["Quantity (g)"]
obj_df = pd.DataFrame({"Cost (€)": {"Cost (€)": sol["primal objective"]}})
display(sol_df)
display(obj_df)

,Quantity (g)
Meat,100.00
Pasta,258.90
TomatoSauce,258.90
OliveOil,10.01


,Cost (€)
Cost (€),18.75


#### Second Solution
We now have both pasta and pasta sauce in our meal. We can see that the cost of the first meal (without sauce) is significantly higher than the second one given the constraint that we imposed (pasta sauce significantly more expensive than pasta for lower amount of glucids), but it would probably taste better :)